# Data Cleaning

In [1]:
import yaml

import pandas as pd

In [2]:
with open("../config/config.yaml", "r") as file:
    config = yaml.safe_load(file)
config

{'raw_data_paths': {'daily': '../data/raw/CBS_2021-2023_Daily_Weather.csv',
  'hourly': '../data/raw/CBS_2021-2023_Hourly_Weather.csv',
  'full': '../data/raw/CBS_2021-2023_Full.csv'},
 'processed_data_paths': {'data_daily': '../data/processed/daily_data.parquet',
  'data_hourly': '../data/processed/hourly_data.parquet'}}

# Load

In [5]:
data_daily  = pd.read_csv(config['raw_data_paths']['daily'])

In [49]:
# import csv

# weathercodes = pd.read_csv('../data/raw/WMO2011h.csv', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC, on_bad_lines='warn')
# weathercodes

In [63]:
data_daily .head()

,Casual,Date,Member,Total_rides,apparent_temperature_mean (°C),temperature_2m_mean (°C),weathercode (wmo code),windspeed_10m_max (km/h)
0,371,2021-01-01,590,961,-0.9,2.3,63,10.6
1,2874,2021-01-02,2844,5718,4.8,7.6,61,19.1
2,495,2021-01-03,1133,1628,0.9,3.9,63,13.0
3,907,2021-01-04,2074,2981,1.0,4.1,3,13.3
4,1057,2021-01-05,2004,3061,1.4,4.0,51,9.2


In [64]:
data_hourly.head()

,Casual,Date,Hour,Member,Total_rides,relativehumidity_2m (%),temperature_2m (°C),weathercode (wmo code),windspeed_10m (km/h)
0,20.0,2021-01-01,0,26.0,46.0,90,2.4,1,10.2
1,23.0,2021-01-01,1,23.0,46.0,88,1.8,1,10.0
2,20.0,2021-01-01,2,28.0,48.0,87,1.1,2,10.1
3,9.0,2021-01-01,3,7.0,16.0,81,1.6,2,10.6
4,7.0,2021-01-01,4,5.0,12.0,77,1.6,2,9.9


# Data Cleaning

In [ ]:
# data_daily.Date = pd.to_datetime(data_daily.Date)
data_daily.Date = data_daily.Date.values.astype('datetime64[D]')

In [55]:
rename_col = {
    'apparent_temperature_mean (°C)':'apparent_temperature_mean',
    'temperature_2m_mean (°C)'      :'temperature_2m_mean',
    'weathercode (wmo code)'        :'wmo_code',
    'windspeed_10m_max (km/h)'      :'windspeed_10m_max'
    }
data_daily = data_daily.rename(columns=rename_col)
data_daily = data_daily.rename(columns=lambda x: x.lower())

# Info, Head, Describe 

In [56]:
data_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype        
---  ------                     --------------  -----        
 0   casual                     1095 non-null   int64        
 1   date                       1095 non-null   datetime64[s]
 2   member                     1095 non-null   int64        
 3   total_rides                1095 non-null   int64        
 4   apparent_temperature_mean  1095 non-null   float64      
 5   temperature_2m_mean        1095 non-null   float64      
 6   wmo_code                   1095 non-null   int64        
 7   windspeed_10m_max          1095 non-null   float64      
dtypes: datetime64[s](1), float64(3), int64(4)
memory usage: 68.6 KB


In [52]:
data_daily.head()

,casual,date,member,total_rides,apparent_temperature_mean,temperature_2m_mean,wmo_code,windspeed_10m_max
0,371,2021-01-01,590,961,-0.9,2.3,63,10.6
1,2874,2021-01-02,2844,5718,4.8,7.6,61,19.1
2,495,2021-01-03,1133,1628,0.9,3.9,63,13.0
3,907,2021-01-04,2074,2981,1.0,4.1,3,13.3
4,1057,2021-01-05,2004,3061,1.4,4.0,51,9.2


In [53]:
data_daily.describe()

,casual,date,member,total_rides,apparent_temperature_mean,temperature_2m_mean,wmo_code,windspeed_10m_max
count,1095.000000,1095,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000
mean,3836.715068,2022-07-02 00:00:00,5929.492237,9766.207306,12.836438,14.412785,24.391781,16.051050
min,174.000000,2021-01-01 00:00:00,323.000000,497.000000,-15.200000,-9.100000,0.000000,5.600000
25%,2122.000000,2021-10-01 12:00:00,4292.500000,6741.500000,2.700000,6.250000,1.000000,11.700000
50%,3595.000000,2022-07-02 00:00:00,5848.000000,9759.000000,13.100000,14.900000,3.000000,14.700000
75%,5321.000000,2023-04-01 12:00:00,7577.000000,13020.000000,23.400000,22.750000,53.000000,19.400000
max,12839.000000,2023-12-31 00:00:00,12580.000000,20174.000000,34.700000,34.700000,75.000000,40.100000
std,2114.895305,NaN,2419.273171,4129.630135,11.889420,9.667675,27.774440,5.880358


# Export / Variable sharing

In [57]:
%store data_daily

Stored 'data_daily' (DataFrame)


In [58]:
data_daily.to_parquet('../data/processed/CBS_2021-2023_Daily_Weather.parquet')